## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [5]:
import sys
print(sys.modules.keys())
print(sys.path)



dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_distutils_hack', 'site', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib._abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', 'types', '_functools', 'functools', '_weakrefset', 'threading', 'importlib.util', 'runpy', 'enum', '_sre', 're._constants', 're._parser', 're._casefix', 're._compiler', 'copyreg', 're', 'fnmatch', 'ntpath', 'errno', 'urllib', 'math', 'ipaddress', 'urllib.parse', 'pathlib', 'collections.abc', 'contextlib', '_typing', 'typing.io', 'typing.re', 'typing', 'ipykernel._ver

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
# import requred modules
import sys
import os
import time
import numpy as np

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import preprocess_sim2real_dataset, load_scenario_npy, compute_and_save_burn_maps_sim2real_dataset
from Strategy import RandomDroneRoutingStrategy, return_no_custom_parameters, SensorPlacementOptimization, RandomSensorPlacementStrategy, LoggedOptimizationSensorPlacementStrategy,DroneRoutingOptimizationSlow, DroneRoutingOptimizationModelReuse, DroneRoutingOptimizationModelReuseIndex, LoggedDroneRoutingStrategy, LogWrapperDrone, LogWrapperSensor
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential, benchmark_on_sim2real_dataset, get_burnmap_parameters,run_benchmark_scenarii_sequential_precompute, benchmark_on_sim2real_dataset_precompute
from displays import create_scenario_video

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Initializing the Julia session. This can take up to 1 minute.
initializing the ground sensor julia module
installing packages
initializing the drone julia module
Julia session initialized.
=== TEST PRINT: Entered RandomPlacementStrategy class definition ===
=== TEST PRINT: Entered DroneRoutingOptimizationModelReuseIndex class definition ===


Uncomment the cell below to preprocess the dataset.
- `n_max_scenarii_per_layout` controls the number of scenarios we convert from jpeg to NumPy files for each layout.
- If executed on all ~50 layouts of the dataset, the code below takes ~15mins to run and generates **400 GB** of data. Make sure to have the space.

In [2]:
# preprocess_sim2real_dataset("WideDataset/", n_max_scenarii_per_layout=100) 

We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [2]:
# change values here to change benchmarking parameters
def my_automatic_layout_parameters(scenario:np.ndarray):
    return {
        "N": scenario.shape[1],
        "M": scenario.shape[2],
        "max_battery_distance": 10,
        "max_battery_time": 10,
        "n_drones": 5,
        "n_ground_stations": 10,
        "n_charging_stations": 5,
    }

In [3]:
# That's very fast to run!
print("starting benchmark")
time_start = time.time()
scenario = load_scenario_npy("Dataset/0001/scenarii/0001_00002.npy")
results, (position_history, ground, charging)  = run_benchmark_scenario(scenario, LogWrapperSensor, LogWrapperDrone, custom_initialization_parameters = {"burnmap_filename": "./Dataset/0001/burn_map.npy", "load_from_logfile": True, "reevaluation_step": 5, "optimization_horizon":5, "strategy_drone": DroneRoutingOptimizationModelReuseIndex, "strategy_sensor": SensorPlacementOptimization}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print(results)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")




starting benchmark
=== TEST PRINT: Entered run_benchmark_scenario ===
Charging Stations (Python, after conversion): {(6, 127), (40, 177), (115, 48), (112, 161), (23, 266)}
Type of charging_stations_locations:  <class 'set'>

DEBUG: Charging Stations passed to Routing Strategy: {(6, 127), (40, 177), (115, 48), (112, 161), (23, 266)}
Type of charging_stations_locations: <class 'set'>

DEBUG: Initial Drone Locations: [['fly', [6, 127]], ['fly', [6, 127]], ['fly', [6, 127]], ['fly', [6, 127]], ['fly', [6, 127]]]
{'delta_t': 58, 'device': 'charging station', 'avg_execution_time': 4.275091763200431e-07, 'fire_size_cells': 788, 'fire_size_percentage': 2.366934999399255, 'percentage_map_explored': 0.09311546317433618, 'total_distance_traveled': 380, 'avg_drone_entropy': 1.4870772396248368, 'sensor_entropy': 2.3025850929940455}
Time taken to run benchmark on the scenario: 0.01964116096496582 seconds


In [8]:
def custom_initialization_parameters_function(input_dir:str):
    print(f"input_dir: {input_dir}")
    return {"burnmap_filename": f"{'/'.join(input_dir.strip('/').split('/')[:-1])}/burn_map.npy", "reevaluation_step": 5, "optimization_horizon":5, "strategy_drone": DroneRoutingOptimizationModelReuseIndex, "strategy_sensor": RandomSensorPlacementStrategy}
#run_benchmark_scenarii_sequential_precompute(input_dir="Dataset/0001/scenarii/", sensor_placement_strategy=LogWrapperSensor, drone_routing_strategy=LogWrapperDrone, custom_initialization_parameters_function=custom_initialization_parameters_function, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=100)
benchmark_on_sim2real_dataset_precompute("Dataset/", LogWrapperSensor, LogWrapperDrone, custom_initialization_parameters_function, return_no_custom_parameters, max_n_scenarii=100, starting_time=0)
    


 --- 
 Processing layout Dataset/0012_02094
input_dir: Dataset/0012_02094/scenarii/
RandomSensorPlacementStrategy


JuliaError: Error loading burn map: ErrorException("parsing header failed: unsupported type O")
Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] load_burn_map(filename::String)
   @ Main ~/Documents/GitHub/wildfire_drone_routing/julia/helper_functions.jl:13
 [3] create_index_routing_model(risk_pertime_file::String, n_drones::Int64, ChargingStations::PyList{Any}, GroundStations::PyList{Any}, optimization_horizon::Int64, max_battery_time::Int64)
   @ Main ~/Documents/GitHub/wildfire_drone_routing/julia/drone_routing_opt.jl:757
 [4] pyjlany_call(self::typeof(create_index_routing_model), args_::Py, kwargs_::Py)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/WMWY0/src/JlWrap/any.jl:47
 [5] _pyjl_callmethod(f::Any, self_::Ptr{PythonCall.C.PyObject}, args_::Ptr{PythonCall.C.PyObject}, nargs::Int64)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/WMWY0/src/JlWrap/base.jl:73
 [6] _pyjl_callmethod(o::Ptr{PythonCall.C.PyObject}, args::Ptr{PythonCall.C.PyObject})
   @ Python

In [9]:
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

Video saved at: display_test_simulation/test_simulation.mp4


We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [4]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

calling julia optimization model
In julia fuinction
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Took 8.259267374873161 seconds
optimization finished
Video saved at: display_test_simulation/test_simulation.mp4


Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [3]:
run_benchmark_scenarii_sequential("MinimalDataset/0001/scenarii/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3)
run_benchmark_scenarii_sequential("MinimalDataset/0001/Satellite_Images_Mask/", RandomSensorPlacementStrategy, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=3, file_format="jpg")

100%|██████████| 3/3 [00:00<00:00, 91.85it/s]


Processing scenario MinimalDataset/0001/scenarii/0001_00033.npy
Processing scenario MinimalDataset/0001/scenarii/0001_00034.npy
Processing scenario MinimalDataset/0001/scenarii/0001_00005.npy
Avg time steps to fire detection: 0.0
Fire found 0.0% of the time by ground sensor
Fire found 0.0% of the time by charging station
Fire found 0.0% of the time by drone
Fire found 100.0% of the time by undetected
Avg routing execution time: 0.0000 sec
Avg fire size at detection (cells): 0.00
Avg fire size at detection (% of map): 0.00%
Avg percentage of map explored by drones: 0.20%
Avg total distance traveled by drones: 357.33 units
Avg drone entropy per timestep: 0.0000
Avg sensor placement entropy: 0.0000


100%|██████████| 3/3 [00:00<00:00, 23.72it/s]

Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00005
Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00002
Processing scenario MinimalDataset/0001/Satellite_Images_Mask/0001_00003
Avg time steps to fire detection: 0.0
Fire found 0.0% of the time by ground sensor
Fire found 0.0% of the time by charging station
Fire found 0.0% of the time by drone
Fire found 100.0% of the time by undetected
Avg routing execution time: 0.0000 sec
Avg fire size at detection (cells): 0.00
Avg fire size at detection (% of map): 0.00%
Avg percentage of map explored by drones: 0.20%
Avg total distance traveled by drones: 373.33 units
Avg drone entropy per timestep: 0.0000
Avg sensor placement entropy: 0.0000


On all of the dataset!

In [ ]:
benchmark_on_sim2real_dataset("WideDataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)

In [4]:
# That's very fast to run!
time_start = time.time()
scenario = load_scenario_npy("WideDataset/0001/scenarii/0001_00002.npy")
device, delta_t, _ = run_benchmark_scenario(scenario, RandomSensorPlacementStrategy, DroneRoutingOptimizationSlow, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "call_every_n_steps": 5, "optimization_horizon": 10}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling ju

In [13]:
# define function to get parameters for optimal drone routing strategy
# def get_routing_optimization_parameters(input_dir:str):
#     return {
#         "burnmap_filename": f"{input_dir}/burn_map.npy",
#         "call_every_n_steps": 5,
#         "optimization_horizon": 10
#     }

# add the log
